In [2]:
from jinja2 import Template

# your template (assumed already in this variable)
new_chat_template = """
{{ bos_token }}

{% if messages[0]['role'] == 'system' %}
    {% if messages[0]['content'] is string %}
        {% set first_user_prefix = messages[0]['content'] + '\n\n' %}
    {% else %}
        {% set first_user_prefix = messages[0]['content'][0]['text'] + '\n\n' %}
    {% endif %}
    {% set loop_messages = messages[1:] %}
{% else %}
    {% set first_user_prefix = "" %}
    {% set loop_messages     = messages %}
{% endif %}

{# Only render the very last message #}
{% set message = loop_messages[-1] %}

{# Normalize assistant role to "model" #}
{% if message['role'] == 'assistant' %}
    {% set role = "model" %}
{% else %}
    {% set role = message['role'] %}
{% endif %}

{{ '<start_of_turn>' + role + '\n' + first_user_prefix }}

{% if message['content'] is string %}
    {{ message['content'] | trim }}
{% elif message['content'] is iterable %}
    {% for item in message['content'] %}
        {% if item['type'] == 'image' %}
            {{ '<start_of_image>' }}
        {% elif item['type'] == 'text' %}
            {{ item['text'] | trim }}
        {% endif %}
    {% endfor %}
{% else %}
    {{ raise_exception("Invalid content type") }}
{% endif %}

{{ '<end_of_turn>\n' }}

{% if add_generation_prompt %}
    {{ '<start_of_turn>model\n' }}
{% endif %}
"""

# test input
messages = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": "Hi Phil, can you please send food and water as soon as possible?",
    }]
}]

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Event A is on 2025-04-23. Please prepare accordingly."
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Acknowledged. I will send an update by 2025-04-24."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Final reminder: deadline is 2025-04-25. Submit all materials before then."
            }
        ]
    }
]


# render
template = Template(new_chat_template)
output = template.render(
    bos_token="<BOS>",
    messages=messages,
    add_generation_prompt=False
)

print(output)



<BOS>


    
    







    


<start_of_turn>user



    
        
            Final reminder: deadline is 2025-04-25. Submit all materials before then.
        
    


<end_of_turn>



